<a href="https://colab.research.google.com/github/btlgs2000/Elettric80/blob/main/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install optuna

In [ ]:
 # fashion mnist

# rete migliore come accuracy
# dropout rate (0, 0.5)
# numero di strati nascosti (1, 5)
# numero di unità per ogni strato nascosto (10, 200)
# relu e sigmoide
# learning rate 10-3 e 10-2 #Adam

from tensorflow.keras import Model, Sequential
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten

import numpy as np

import optuna

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
x_train = (x_train / 255).astype(np.float32)
x_test = (x_test / 255).astype(np.float32)

In [ ]:
def get_model(droput_rate:float, num_hidden:int, num_units:int, activation:str) -> Model:
    ''' Restituisce un modello '''
    model = Sequential([Flatten()])

    # strati nascosti
    for _ in range(num_hidden):
        model.add(Dense(num_units, activation))

    # ultimo strato
    model.add(Dense(10, 'softmax'))

    return model

In [ ]:
class Objective:
    def __init__(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    def __call__(self, trial):
        # scelta iperparametri
        # trial.suggest....
        num_hidden = trial.suggest_int('num_hidden', 1, 5)
        num_units = trial.suggest_int('num_units', 10, 200)
        dropout_rate = trial.suggest_uniform('dropout_rate', 0., 0.5)
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-2)
        activation = trial.suggest_categorical('activation', ['relu', 'sigmoid'])
        
        # get_mode
        model = get_model(dropout_rate, num_hidden, num_units, activation)
        model.compile(optimizer=Adam(learning_rate), loss='sparse_categorical_crossentropy', metrics='accuracy')
        
        # addestramento # 10 epoche con early stopping su accuratezza sul validation, patience=2
        model.fit(x=self.x_train, y=self.y_train, epochs=10, callbacks=[EarlyStopping(monitor='val_accuracy', patience=2)], validation_split=0.2)

        return max(model.history.history['val_accuracy'])
        # ritorna l'accuracy massima

In [ ]:
study = optuna.create_study(storage='sqlite:///model_selection.db', study_name='model_selection', direction='maximize')

In [ ]:
study.optimize(Objective(x_train, y_train), n_trials=200)